In [7]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import mapclassify as mc
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap

In [8]:
modelFolderPath = "Accessibility_Dissolve_Animation"

In [9]:
chicagoNameList = glob.glob(os.path.join(modelFolderPath,"Chicago_ACC.*.shp"))
illinoisNameList = glob.glob(os.path.join(modelFolderPath,"Illinois_ACC.*.shp"))
chicagoList = []
illinoisList = []

for i in range(0, len(chicagoNameList)):
    chicagoList.append(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC.*.shp"))[i][45:55])
for i in range(0, len(illinoisNameList)):
    illinoisList.append(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC.*.shp"))[i][46:56])
    
print(chicagoList)
print(illinoisList)

['06-24-2020', '06-25-2020']
['06-25-2020']


In [10]:
def selectFirstDate(nameList):
    dmin = datetime.strptime(nameList[0], "%m-%d-%Y").date()
    index = 0
    for i in range(0, len(nameList)):
        d = datetime.strptime(nameList[i], "%m-%d-%Y").date()
        if (d<=dmin):
            index = i
            dmin = d
    return index

def selectLastDate(nameList):
    dmax = datetime.strptime(nameList[0], "%m-%d-%Y").date()
    index = 0
    for i in range(0, len(nameList)):
        d = datetime.strptime(nameList[i], "%m-%d-%Y").date()
        if (d>=dmax):
            index = i
            dmax = d
    return index

In [11]:
firstIndex = selectFirstDate(chicagoList)
lastIndex = selectLastDate(chicagoList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[lastIndex]+"*.shp"))[0])
nbArray_i = mc.NaturalBreaks(df['hospital_i'], k=6).bins.astype(float)

def classifyChicago(value):
    if value >= 0 and value <= nbArray_i[0]:
        return 0
    if value >= nbArray_i[0] and value <= nbArray_i[1]:
        return 1
    elif value > nbArray_i[1] and value <= nbArray_i[2]:
        return 2
    elif value > nbArray_i[2] and value <= nbArray_i[3]:
        return 3
    elif value > nbArray_i[3] and value <= nbArray_i[4]:
        return 4
    else:
        return 5

def classRangeChicago(value):
    if value == 0:
        return "0~"+str(round(nbArray_i[0],2))
    if value == 1:
        return str(round(nbArray_i[0],2))+"~"+str(round(nbArray_i[1],2))
    elif value == 2:
        return str(round(nbArray_i[1],2))+"~"+str(round(nbArray_i[2],2))
    elif value == 3:
        return str(round(nbArray_i[2],2))+"~"+str(round(nbArray_i[3],2))
    elif value == 4:
        return str(round(nbArray_i[3],2))+"~"+str(round(nbArray_i[4],2))
    else:
        return str(round(nbArray_i[4],2))+"~"+str(round(nbArray_i[5],2))

chicagoACC_dissolve_list = []    

for i in range(0, len(chicagoList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (chicagoList[i] == chicagoList[i-1]):
            continue
    chicagoACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[i]+"*.shp"))[0])
    #chicagoACC = chicagoACC.rename(columns={"normal_cov": "normal_val"})
    
    chicagoACC['category'] = chicagoACC.apply(lambda x: classifyChicago(x.hospital_i), axis = 1)
    chicagoACC_dissolve = chicagoACC.dissolve(by='category', aggfunc = 'mean')
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['id'])
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['hospital_v'])
    chicagoACC_dissolve["category"] = range(0, len(chicagoACC_dissolve))
    chicagoACC_dissolve['date'] = datetime.strptime(chicagoList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    chicagoACC_dissolve['range'] = chicagoACC_dissolve.apply(lambda x: classRangeChicago(x.category), axis = 1)
    
    chicagoACC_dissolve = chicagoACC_dissolve.to_crs({'init': 'epsg:4326'})
    #chicagoACC_dissolve.to_file("Chicago_ACC."+chicagoList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    chicagoACC_dissolve_list.append(chicagoACC_dissolve)
    
chicagoACC_concat = pd.concat(chicagoACC_dissolve_list, axis=0)
chicagoACC_concat = chicagoACC_concat.drop(columns=['category'])
chicagoACC_concat = chicagoACC_concat.reset_index()

#delta = timedelta(days=len(chicagoList))
firstDate = datetime.strptime(chicagoList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(chicagoList[lastIndex], "%m-%d-%Y").date()
chicagoACC_concat["dt_start"] = firstDate.strftime("%Y-%m-%d") #(lastDate-delta).strftime("%Y-%m-%d")
chicagoACC_concat["dt_end"] = lastDate.strftime("%Y-%m-%d")
chicagoACC_concat["dt_unit"] = "day"
chicagoACC_concat["cases_ts"] = "0,0,0,0,0,0,0"#str(np.zeros(len(chicagoList),int)).replace(" ",",")
chicagoACC_concat.to_file("Chicago_ACC_i.geojson", driver="GeoJSON")

print("done")

done


In [12]:
chicagoACC_concat

,category,geometry,hospital_i,date,range,dt_start,dt_end,dt_unit,cases_ts
0,0,"MULTIPOLYGON (((-87.52838 41.64517, -87.52666 ...",0.058372,2020-06-24,0~0.21,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
1,1,"MULTIPOLYGON (((-87.59774 41.83622, -87.59602 ...",0.369879,2020-06-24,0.21~0.45,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
2,2,"MULTIPOLYGON (((-87.64067 41.95758, -87.63895 ...",0.531566,2020-06-24,0.45~0.59,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
3,3,"MULTIPOLYGON (((-87.71725 41.79726, -87.71554 ...",0.655139,2020-06-24,0.59~0.71,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
4,4,"MULTIPOLYGON (((-87.63703 41.76621, -87.63356 ...",0.767892,2020-06-24,0.71~0.83,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
5,5,"MULTIPOLYGON (((-87.74868 41.81057, -87.74697 ...",0.888180,2020-06-24,0.83~1.0,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
6,0,"MULTIPOLYGON (((-87.52838 41.64517, -87.52666 ...",0.058372,2020-06-25,0~0.21,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
7,1,"MULTIPOLYGON (((-87.59774 41.83622, -87.59602 ...",0.369879,2020-06-25,0.21~0.45,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
8,2,"MULTIPOLYGON (((-87.64067 41.95758, -87.63895 ...",0.531566,2020-06-25,0.45~0.59,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
9,3,"MULTIPOLYGON (((-87.71725 41.79726, -87.71554 ...",0.655139,2020-06-25,0.59~0.71,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"


In [13]:
firstIndex = selectFirstDate(chicagoList)
lastIndex = selectLastDate(chicagoList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[lastIndex]+"*.shp"))[0])
nbArray_v = mc.NaturalBreaks(df['hospital_v'], k=6).bins.astype(float)

def classifyChicago(value):
    if value >= 0 and value <= nbArray_v[0]:
        return 0
    if value >= nbArray_v[0] and value <= nbArray_v[1]:
        return 1
    elif value > nbArray_v[1] and value <= nbArray_v[2]:
        return 2
    elif value > nbArray_v[2] and value <= nbArray_v[3]:
        return 3
    elif value > nbArray_v[3] and value <= nbArray_v[4]:
        return 4
    else:
        return 5

def classRangeChicago(value):
    if value == 0:
        return "0~"+str(round(nbArray_v[0],2))
    if value == 1:
        return str(round(nbArray_v[0],2))+"~"+str(round(nbArray_v[1],2))
    elif value == 2:
        return str(round(nbArray_v[1],2))+"~"+str(round(nbArray_v[2],2))
    elif value == 3:
        return str(round(nbArray_v[2],2))+"~"+str(round(nbArray_v[3],2))
    elif value == 4:
        return str(round(nbArray_v[3],2))+"~"+str(round(nbArray_v[4],2))
    else:
        return str(round(nbArray_v[4],2))+"~"+str(round(nbArray_v[5],2))

chicagoACC_dissolve_list = []    

for i in range(0, len(chicagoList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (chicagoList[i] == chicagoList[i-1]):
            continue
    chicagoACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Chicago_ACC."+chicagoList[i]+"*.shp"))[0])
    #chicagoACC = chicagoACC.rename(columns={"normal_cov": "normal_val"})
    
    chicagoACC['category'] = chicagoACC.apply(lambda x: classifyChicago(x.hospital_v), axis = 1)
    chicagoACC_dissolve = chicagoACC.dissolve(by='category', aggfunc = 'mean')
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['id'])
    chicagoACC_dissolve = chicagoACC_dissolve.drop(columns=['hospital_i'])
    chicagoACC_dissolve["category"] = range(0, len(chicagoACC_dissolve))
    chicagoACC_dissolve['date'] = datetime.strptime(chicagoList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    chicagoACC_dissolve['range'] = chicagoACC_dissolve.apply(lambda x: classRangeChicago(x.category), axis = 1)
    
    chicagoACC_dissolve = chicagoACC_dissolve.to_crs({'init': 'epsg:4326'})
    #chicagoACC_dissolve.to_file("Chicago_ACC."+chicagoList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    chicagoACC_dissolve_list.append(chicagoACC_dissolve)
    
chicagoACC_concat = pd.concat(chicagoACC_dissolve_list, axis=0)
chicagoACC_concat = chicagoACC_concat.drop(columns=['category'])
chicagoACC_concat = chicagoACC_concat.reset_index()

#delta = timedelta(days=len(chicagoList))
firstDate = datetime.strptime(chicagoList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(chicagoList[lastIndex], "%m-%d-%Y").date()
chicagoACC_concat["dt_start"] = firstDate.strftime("%Y-%m-%d") #(lastDate-delta).strftime("%Y-%m-%d")
chicagoACC_concat["dt_end"] = lastDate.strftime("%Y-%m-%d")
chicagoACC_concat["dt_unit"] = "day"
chicagoACC_concat["cases_ts"] = "0,0,0,0,0,0,0"#str(np.zeros(len(chicagoList),int)).replace(" ",",")
chicagoACC_concat.to_file("Chicago_ACC_v.geojson", driver="GeoJSON")

print("done")

done


In [14]:
chicagoACC_concat

,category,geometry,hospital_v,date,range,dt_start,dt_end,dt_unit,cases_ts
0,0,"MULTIPOLYGON (((-87.52838 41.64517, -87.52666 ...",0.058577,2020-06-24,0~0.21,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
1,1,"MULTIPOLYGON (((-87.60877 41.89921, -87.60705 ...",0.374463,2020-06-24,0.21~0.46,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
2,2,"MULTIPOLYGON (((-87.74199 41.68227, -87.74028 ...",0.539944,2020-06-24,0.46~0.6,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
3,3,"MULTIPOLYGON (((-87.70545 41.67124, -87.70369 ...",0.667218,2020-06-24,0.6~0.72,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
4,4,"MULTIPOLYGON (((-87.71680 41.75673, -87.71504 ...",0.776608,2020-06-24,0.72~0.83,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
5,5,"MULTIPOLYGON (((-87.66018 41.82237, -87.65847 ...",0.889519,2020-06-24,0.83~1.0,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
6,0,"MULTIPOLYGON (((-87.52838 41.64517, -87.52666 ...",0.058577,2020-06-25,0~0.21,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
7,1,"MULTIPOLYGON (((-87.60877 41.89921, -87.60705 ...",0.374463,2020-06-25,0.21~0.46,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
8,2,"MULTIPOLYGON (((-87.74199 41.68227, -87.74028 ...",0.539944,2020-06-25,0.46~0.6,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"
9,3,"MULTIPOLYGON (((-87.70545 41.67124, -87.70369 ...",0.667218,2020-06-25,0.6~0.72,2020-06-24,2020-06-25,day,"0,0,0,0,0,0,0"


In [15]:
firstIndex = selectFirstDate(illinoisList)
lastIndex = selectLastDate(illinoisList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[lastIndex]+"*.shp"))[0])
nbArray_i = mc.NaturalBreaks(df['hospital_i'], k=6).bins.astype(float)

def classifyIllinois(value):   
    if value >= 0 and value <= nbArray_i[0]:
        return 0
    if value >= nbArray_i[0] and value <= nbArray_i[1]:
        return 1
    elif value > nbArray_i[1] and value <= nbArray_i[2]:
        return 2
    elif value > nbArray_i[2] and value <= nbArray_i[3]:
        return 3
    elif value > nbArray_i[3] and value <= nbArray_i[4]:
        return 4
    else:
        return 5
    
def classRangeIllinois(value):
    if value == 0:
        return "0~"+str(round(nbArray_i[0],2))
    if value == 1:
        return str(round(nbArray_i[0],2))+"~"+str(round(nbArray_i[1],2))
    elif value == 2:
        return str(round(nbArray_i[1],2))+"~"+str(round(nbArray_i[2],2))
    elif value == 3:
        return str(round(nbArray_i[2],2))+"~"+str(round(nbArray_i[3],2))
    elif value == 4:
        return str(round(nbArray_i[3],2))+"~"+str(round(nbArray_i[4],2))
    else:
        return str(round(nbArray_i[4],2))+"~"+str(round(nbArray_i[5],2))

illinoisACC_dissolve_list = []    

for i in range(0, len(illinoisList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (illinoisList[i] == illinoisList[i-1]):
            continue
    illinoisACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[i]+"*.shp"))[0])
    #illinoisACC = illinoisACC.rename(columns={"normal_cov": "normal_val"})
    
    illinoisACC['category'] = illinoisACC.apply(lambda x: classifyIllinois(x.hospital_i), axis = 1)
    illinoisACC_dissolve = illinoisACC.dissolve(by='category', aggfunc = 'mean')
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['id'])
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['hospital_v'])
    illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    illinoisACC_dissolve['date'] = datetime.strptime(illinoisList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    illinoisACC_dissolve['range'] = illinoisACC_dissolve.apply(lambda x: classRangeIllinois(x.category), axis = 1)
    #illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    
    illinoisACC_dissolve = illinoisACC_dissolve.to_crs({'init': 'epsg:4326'})
    #illinoisACC_dissolve.to_file("Illinois_ACC."+illinoisList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    illinoisACC_dissolve_list.append(illinoisACC_dissolve)
    
illinoisACC_concat = pd.concat(illinoisACC_dissolve_list, axis=0)
illinoisACC_concat = illinoisACC_concat.drop(columns=['category'])
illinoisACC_concat = illinoisACC_concat.reset_index()

#delta = timedelta(days=len(illinoisList))
firstDate = datetime.strptime(illinoisList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(illinoisList[lastIndex], "%m-%d-%Y").date()
illinoisACC_concat["dt_start"] = firstDate.strftime("%Y-%m-%d") #(lastDate-delta).strftime("%Y-%m-%d")
illinoisACC_concat["dt_end"] = lastDate.strftime("%Y-%m-%d")
illinoisACC_concat["dt_unit"] = "day"
illinoisACC_concat["cases_ts"] = "0,0,0,0,0,0,0"#str(np.zeros(len(illinoisList),int)).replace(" ",",")
illinoisACC_concat.to_file("Illinois_ACC_i.geojson", driver="GeoJSON")

print("done")

done


In [16]:
illinoisACC_concat

,category,geometry,hospital_i,date,range,dt_start,dt_end,dt_unit,cases_ts
0,0,"MULTIPOLYGON (((-88.43522 37.07011, -88.45102 ...",0.003575,2020-06-25,0~0.02,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
1,1,"MULTIPOLYGON (((-88.98350 39.36009, -88.99961 ...",0.035645,2020-06-25,0.02~0.07,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
2,2,"MULTIPOLYGON (((-90.75191 41.07270, -90.73604 ...",0.110517,2020-06-25,0.07~0.15,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
3,3,"MULTIPOLYGON (((-88.46411 37.72320, -88.44730 ...",0.197737,2020-06-25,0.15~0.32,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
4,4,"POLYGON ((-88.34192 39.09894, -88.32481 39.076...",0.555327,2020-06-25,0.32~0.58,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
5,5,"POLYGON ((-88.69178 39.07186, -88.67564 39.094...",0.992674,2020-06-25,0.58~1.0,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"


In [17]:
firstIndex = selectFirstDate(illinoisList)
lastIndex = selectLastDate(illinoisList)
df = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[lastIndex]+"*.shp"))[0])
nbArray_v = mc.NaturalBreaks(df['hospital_v'], k=6).bins.astype(float)

def classifyIllinois(value):   
    if value >= 0 and value <= nbArray_v[0]:
        return 0
    if value >= nbArray_v[0] and value <= nbArray_v[1]:
        return 1
    elif value > nbArray_v[1] and value <= nbArray_v[2]:
        return 2
    elif value > nbArray_v[2] and value <= nbArray_v[3]:
        return 3
    elif value > nbArray_v[3] and value <= nbArray_v[4]:
        return 4
    else:
        return 5
    
def classRangeIllinois(value):
    if value == 0:
        return "0~"+str(round(nbArray_v[0],2))
    if value == 1:
        return str(round(nbArray_v[0],2))+"~"+str(round(nbArray_v[1],2))
    elif value == 2:
        return str(round(nbArray_v[1],2))+"~"+str(round(nbArray_v[2],2))
    elif value == 3:
        return str(round(nbArray_v[2],2))+"~"+str(round(nbArray_v[3],2))
    elif value == 4:
        return str(round(nbArray_v[3],2))+"~"+str(round(nbArray_v[4],2))
    else:
        return str(round(nbArray_v[4],2))+"~"+str(round(nbArray_v[5],2))

illinoisACC_dissolve_list = []    

for i in range(0, len(illinoisList)):
    #In case we have multiple files for one day
    if (i != 0):
        if (illinoisList[i] == illinoisList[i-1]):
            continue
    illinoisACC = gpd.read_file(glob.glob(os.path.join(modelFolderPath,"Illinois_ACC."+illinoisList[i]+"*.shp"))[0])
    #illinoisACC = illinoisACC.rename(columns={"normal_cov": "normal_val"})
    
    illinoisACC['category'] = illinoisACC.apply(lambda x: classifyIllinois(x.hospital_v), axis = 1)
    illinoisACC_dissolve = illinoisACC.dissolve(by='category', aggfunc = 'mean')
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['id'])
    illinoisACC_dissolve = illinoisACC_dissolve.drop(columns=['hospital_i'])
    illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    illinoisACC_dissolve['date'] = datetime.strptime(illinoisList[i], "%m-%d-%Y").date().strftime("%Y-%m-%d")
    illinoisACC_dissolve['range'] = illinoisACC_dissolve.apply(lambda x: classRangeIllinois(x.category), axis = 1)
    #illinoisACC_dissolve["category"] = range(0, len(illinoisACC_dissolve))
    
    illinoisACC_dissolve = illinoisACC_dissolve.to_crs({'init': 'epsg:4326'})
    #illinoisACC_dissolve.to_file("Illinois_ACC."+illinoisList[i]+"_dissolve.geojson", driver="GeoJSON")
    
    illinoisACC_dissolve_list.append(illinoisACC_dissolve)
    
illinoisACC_concat = pd.concat(illinoisACC_dissolve_list, axis=0)
illinoisACC_concat = illinoisACC_concat.drop(columns=['category'])
illinoisACC_concat = illinoisACC_concat.reset_index()

#delta = timedelta(days=len(illinoisList))
firstDate = datetime.strptime(illinoisList[firstIndex], "%m-%d-%Y").date()
lastDate = datetime.strptime(illinoisList[lastIndex], "%m-%d-%Y").date()
illinoisACC_concat["dt_start"] = firstDate.strftime("%Y-%m-%d") #(lastDate-delta).strftime("%Y-%m-%d")
illinoisACC_concat["dt_end"] = lastDate.strftime("%Y-%m-%d")
illinoisACC_concat["dt_unit"] = "day"
illinoisACC_concat["cases_ts"] = "0,0,0,0,0,0,0"#str(np.zeros(len(illinoisList),int)).replace(" ",",")
illinoisACC_concat.to_file("Illinois_ACC_v.geojson", driver="GeoJSON")

print("done")

done


In [18]:
illinoisACC_concat

,category,geometry,hospital_v,date,range,dt_start,dt_end,dt_unit,cases_ts
0,0,"MULTIPOLYGON (((-88.43522 37.07011, -88.45102 ...",0.003501,2020-06-25,0~0.02,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
1,1,"MULTIPOLYGON (((-88.88304 39.36176, -88.89917 ...",0.036526,2020-06-25,0.02~0.08,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
2,2,"MULTIPOLYGON (((-88.36587 37.72438, -88.34908 ...",0.118369,2020-06-25,0.08~0.15,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
3,3,"MULTIPOLYGON (((-87.58769 38.72220, -87.57091 ...",0.198266,2020-06-25,0.15~0.31,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
4,4,"POLYGON ((-88.34192 39.09894, -88.32481 39.076...",0.555399,2020-06-25,0.31~0.58,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
5,5,"POLYGON ((-88.69178 39.07186, -88.67564 39.094...",0.992933,2020-06-25,0.58~1.0,2020-06-25,2020-06-25,day,"0,0,0,0,0,0,0"
